## Data Science Case Study

The City of Chicago provides a robust set of data assets online here: https://data.cityofchicago.org
Included in this repository is a dataset that includes all of the public libraries in the City of Chicago (feel free to use additional data as you see fit). We’d like you to create a script that will return the closest library as the crow flies to a given location in the city.
Given the parameters of:

- Longitude and lattitude for a point in the city
- Day of week
- Time of day

The script should return, for the five closest libraries, ranked (geographically farthest first):

- Library Name
- Address
- Popularity (calculated by you)
- Currently open or closed

When you are ready, send us all relevant code and documentation, and we’ll schedule an interview in person to walk through it together. While our preferred language is Python, feel free to use which ever language you prefer.


In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#enable IPython to display matplotlib graphs
%matplotlib inline
import csv

## Data Cleansing/EDA

- data source 1: chicago library locations, hours, contact
- data source 2: chicago library visitors in 2016

In [102]:
lib = pd.read_csv('Libraries_-_Locations__Hours_and_Contact_Information.csv')
vis = pd.read_csv('Libraries_-_2016_Visitors_by_Location.csv')

In [103]:
lib.columns

Index(['NAME ', 'HOURS OF OPERATION', 'CYBERNAVIGATOR',
       'TEACHER IN THE LIBRARY', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE',
       'WEBSITE', 'LOCATION'],
      dtype='object')

In [104]:
lib.head(3)

,NAME,HOURS OF OPERATION,CYBERNAVIGATOR,TEACHER IN THE LIBRARY,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,LOCATION
0,Albany Park,"M, W: 10AM-6PM; TU, TH: 12PM-8PM; F, SA: 9AM-...",Yes,Yes,3401 W. Foster Avenue,CHICAGO,IL,60625,(773) 539-5450,https://www.chipublib.org/locations/3/,"(41.975456, -87.71409)"
1,Altgeld,"M, W: 12PM-8PM; TU, TH: 10AM-6PM; F, SA: 9AM-5...",Yes,Yes,13281 S. Corliss Avenue,CHICAGO,IL,60827,(312) 747-3270,https://www.chipublib.org/locations/4/,"(41.65473021837776, -87.6022302609835)"
2,Archer Heights,"M, W: 12PM-8PM; TU, TH: 10AM-6PM; F, SA: 9AM-5...",No,Yes,5055 S. Archer Avenue,CHICAGO,IL,60632,(312) 747-9241,https://www.chipublib.org/locations/5/,"(41.8012136599335, -87.72649071431441)"


In [105]:
lib.shape

(80, 11)

In [106]:
lib['HOURS OF OPERATION'].value_counts()

M, W: 12PM-8PM; TU, TH: 10AM-6PM; F, SA: 9AM-5PM; SU: Closed     39
M, W: 10AM-6PM;  TU, TH: 12PM-8PM; F, SA: 9AM-5PM; SU: Closed    36
M-TH: 9AM-9PM; F, SA: 9AM-5PM; SU: 1PM-5PM                        3
M-TH: 9AM-7PM; F, SA: 9AM-5PM; SU: 1PM-5PM                        1
M-TH: 10AM-6PM; F: 9AM-5PM; SA, SU: Closed                        1
Name: HOURS OF OPERATION, dtype: int64

In [107]:
vis.columns

Index(['LOCATION', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE',
       'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER',
       'YTD'],
      dtype='object')

In [108]:
vis.head(5)

,LOCATION,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD
0,Albany Park,11735.0,10593.0,12368.0,12556.0,10915.0,15341.0,14151.0,13907.0,14119.0,16460.0,14463.0,12789.0,159397.0
1,Altgeld,5335.0,5034.0,4529.0,4471.0,3880.0,4431.0,5526.0,5867.0,5433.0,5064.0,4584.0,3325.0,57479.0
2,Archer Heights*,8375.0,9079.0,10522.0,9747.0,8714.0,9567.0,9514.0,8831.0,8611.0,9209.0,8400.0,6754.0,107323.0
3,Austin,5849.0,5190.0,5691.0,6313.0,6092.0,7489.0,9012.0,8202.0,7521.0,7257.0,6395.0,6632.0,81643.0
4,Austin-Irving,8726.0,9201.0,10017.0,10010.0,8346.0,9571.0,9567.0,9386.0,9110.0,10505.0,10540.0,9913.0,114892.0


In [109]:
vis.shape

(80, 14)

In [110]:
# create decile ranking as metric for library popularity 

vis['popularity'] = pd.qcut(vis['YTD'], 10, labels = False)
vis['popularity'].value_counts()

4.0    8
0.0    8
8.0    8
2.0    8
7.0    8
3.0    8
6.0    8
1.0    8
9.0    8
5.0    7
Name: popularity, dtype: int64

In [111]:
# clear irregular character in vis['LOCATION'] to join table lib on lib['NAME']

vis['LOCATION']=vis['LOCATION'].apply(lambda x: str(x).replace('*','').strip())
vis['LOCATION'].values

lib['NAME ']=lib['NAME '].apply(lambda x: str(x).strip())
lib['NAME '].values

len(set(vis['LOCATION'].tolist()).intersection(set(lib['NAME '].tolist()))) # 78, missing two matches

# find the two missing match from vis to lib:
set(vis['LOCATION'].tolist()).difference(set(lib['NAME '].tolist()))

{'Harold Washington Library Center', 'nan'}

In [112]:
vis[vis['LOCATION'] == 'Harold Washington Library Center']

vis[vis['LOCATION'] == 'nan'] #bad data row, ignore and let remain missing

# the mismatch happens when library name for the Harold Washington is different in two tables
lib[lib['NAME '].str.contains('Harold')] 

# we fix the library name in vis to be consistent with lib:
vis.loc[vis['LOCATION'] == 'Harold Washington Library Center','LOCATION'] = 'Harold Washington-HWLC'

In [113]:
# after library name fix, we expect 79 out 80 total libraries in lib could be joined with vis:

len(set(vis['LOCATION'].tolist()).intersection(set(lib['NAME '].tolist()))) # cleared

79

In [114]:
# also change the column name for library name in vis to be consistent with lib:

lib = lib.rename(columns={'NAME ': 'NAME'})
vis = vis.rename(columns={'LOCATION':'NAME'})

In [115]:
vis.columns

Index(['NAME', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY',
       'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER', 'YTD',
       'popularity'],
      dtype='object')

In [116]:
# merge lib and vis table together:
lib = pd.merge(lib, vis, how = 'left', on = 'NAME')
lib.columns

Index(['NAME', 'HOURS OF OPERATION', 'CYBERNAVIGATOR',
       'TEACHER IN THE LIBRARY', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE',
       'WEBSITE', 'LOCATION', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY',
       'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER',
       'DECEMBER', 'YTD', 'popularity'],
      dtype='object')

In [117]:
lib.shape

(80, 25)

In [118]:
# convert location coordinates into list type:
lib['LOCATION'] = lib['LOCATION'].apply(lambda x: list(map(float, x.replace('(', '').replace(')', '').split(','))))

## Library Quest Function

- input parameter: starting latitute, starting longitute, time of day, day of week, number of closest library to return
- return values: library name, distance (furthers to closest within top n results), address, operation status (open/closed), popularity in terms of number of visitors (deciles 0 to 9 with 0 being least popular and 9 being most popular)


In [119]:
# define closest library calculation function:

def get_closest_lib(lat, lng, time, day, n):
    
    from math import radians, sin, cos, atan2
    from datetime import datetime
    import re
    
    #lib['LOCATION'] = lib['LOCATION'].apply(lambda x: list(map(float, x.replace('(', '').replace(')', '').split(','))))
    
    def get_distance(lat, lng, lat1, lng1):
        R = 6373.0
        lat = radians(lat)
        lng = radians(lng)
        lat1 = radians(lat1)
        lng1 = radians(lng1)

        dlon = lng1 - lng
        dlat = lat1 - lat

        a = sin(dlat / 2)**2 + cos(lat) * cos(lat1) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
    
        return distance
    
    lib['distance'] = lib['LOCATION'].apply(lambda x: get_distance(lat, lng, x[0], x[1]))
    
    top_n_lib = lib.sort_values(by = 'distance', ascending = True)[
        ['NAME','distance', 'ADDRESS','HOURS OF OPERATION', 'popularity']].head(n)
    
    #return top_n_lib
    
    # define library operating function:
    from datetime import datetime

    def get_lib_open_close(time, day):

        list_of_operation_hours = top_n_lib['HOURS OF OPERATION'].tolist()

        current_time = datetime.strptime(time.upper(), '%I%p')

        # check close/open status if day = Monday:
        if day.lower() in ('monday', 'mon', 'm'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'M' in hours.split(':')[0] or 'M-TH' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)
            

        # check close/open status if day = Tuesday:
        elif day.lower() in ('tuesday', 'tues', 'tu'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'TU' in hours.split(':')[0] or 'M-TH' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]
            
            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)
            

        # check close/open status if day = Wednesday:
        elif day.lower() in ('wednesday', 'wed', 'w'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'W' in hours.split(':')[0] or 'M-TH' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)


        # check close/open status if day = Thursday:
        elif day.lower() in ('thursday', 'thur', 'th'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'TH' in hours.split(':')[0] or 'M-TH' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)

        # check close/open status if day = Friday:
        elif day.lower() in ('friday', 'fri', 'f'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'F' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)

        # check close/open status if day = Saturday:
        elif day.lower() in ('saturday', 'sat', 'sa'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                               for hours in this_operation_hours.split(';') 
                               if 'SA' in hours.split(':')[0]]

            list_of_today_hours_start=[re.findall('.+(?=-)', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[re.findall('(?<=-).+', time_range)[0]
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > datetime.strptime(hours_start, '%I%p') 
                                       and current_time < datetime.strptime(hours_close, '%I%p'))
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)

        #check close/open status if day = Sunday:   
        elif day.lower() in ('sunday', 'sun', 'su'):
            list_of_today_hours = [re.findall('(?<=: ).+', hours)[0] for this_operation_hours in list_of_operation_hours
                                       for hours in this_operation_hours.split(';') 
                                       if 'SU' in hours.split(':')[0]]

            list_of_today_hours_start=[datetime.strptime(re.findall('.+(?=-)', time_range)[0],'%I%p') 
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_today_hours_close=[datetime.strptime(re.findall('(?<=-).+', time_range)[0],'%I%p')
                                       if time_range != 'Closed' else time_range
                                       for time_range in list_of_today_hours]

            list_of_current_status = ['open' if today_hours != 'Closed' and
                                      (current_time > hours_start and current_time < hours_close)
                                      else 'Closed'
                                     for today_hours, hours_start, hours_close 
                                      in zip(list_of_today_hours, list_of_today_hours_start,list_of_today_hours_close)]

            return list_of_current_status

            #print(list_of_today_hours)
            #print(list_of_today_hours_start)
            #print(list_of_today_hours_close)
            #print(list_of_current_status)
    
    top_n_lib['library_operation_status']=get_lib_open_close(time, day)
    
    top_n_lib_desc = top_n_lib.sort_values(by='distance', ascending = False)
    return top_n_lib_desc[['NAME', 'distance','ADDRESS','library_operation_status', 'popularity']]

get_closest_lib(41.975456, -87.71409, '2PM', 'Wednesday', 5)

,NAME,distance,ADDRESS,library_operation_status,popularity
53,Northtown,2.811353,6435 N. California Avenue,open,7.0
34,Independence,2.408837,3548 W. Irving Park Road,open,0.0
46,Mayfair,2.141651,4400 W. Lawrence Avenue,open,1.0
13,Budlong Woods,1.690295,5630 N. Lincoln Avenue,open,7.0
0,Albany Park,0.000000,3401 W. Foster Avenue,open,9.0


In [120]:
get_closest_lib(41.975456, -87.71409, '6PM', 'Wednesday', 8)

,NAME,distance,ADDRESS,library_operation_status,popularity
8,Bezazian,4.432463,1226 W. Ainslie Street,open,8.0
35,Jefferson Park,4.021298,5363 W. Lawrence Avenue,Closed,5.0
64,Sulzer Regional,2.851276,4455 N. Lincoln Avenue,open,9.0
53,Northtown,2.811353,6435 N. California Avenue,Closed,7.0
34,Independence,2.408837,3548 W. Irving Park Road,Closed,0.0
46,Mayfair,2.141651,4400 W. Lawrence Avenue,open,1.0
13,Budlong Woods,1.690295,5630 N. Lincoln Avenue,open,7.0
0,Albany Park,0.000000,3401 W. Foster Avenue,Closed,9.0


In [123]:
get_closest_lib(41.975456, -87.71409, '1PM', 'Sunday', 6)

,NAME,distance,ADDRESS,library_operation_status,popularity
64,Sulzer Regional,2.851276,4455 N. Lincoln Avenue,Closed,9.0
53,Northtown,2.811353,6435 N. California Avenue,Closed,7.0
34,Independence,2.408837,3548 W. Irving Park Road,Closed,0.0
46,Mayfair,2.141651,4400 W. Lawrence Avenue,Closed,1.0
13,Budlong Woods,1.690295,5630 N. Lincoln Avenue,Closed,7.0
0,Albany Park,0.000000,3401 W. Foster Avenue,Closed,9.0
